# Scraping-Skript für scilogs.spektrum.de

Autor: Simon Meier-Vieracker (https://github.com/fussballlinguist/)

In [1]:
import requests
import xml.etree.ElementTree as et
from xml.dom import minidom
import re
from bs4 import BeautifulSoup

In [2]:
#HTML laden und parsen
url = "https://scilogs.spektrum.de/engelbart-galaxis/wer-entscheidet-ueber-sprache/"
wfb = requests.get(url)
soup = BeautifulSoup(wfb.content, 'html.parser')

In [3]:
#Metadaten ziehen 
title = soup.title.text
date = soup.time['datetime'][:10]
author = soup.find("span", class_="author").a.text.strip()

#XML anlegen und Metadaten in XML schreiben 
corpus = et.Element("corpus")
text = et.SubElement(corpus, "text", {"title":title, "date":date,"author":author})
main = et.SubElement(text, "main")
comments_element = et.SubElement(text, "comments")

In [4]:
#Text ziehen
main_div = soup.find("div", class_="hentry__content")
ps = main_div.find_all("p")

#Text in XML schreiben
head_element = et.SubElement(main, "head")
head_element.text = soup.h1.text
for p in ps:
    p_element = et.SubElement(main, "p")
    p_element.text = p.text.strip()

#Kommentare ziehen
comments_section = soup.find("div", id="comments")
comments = comments_section.find_all("article", class_="comment regular")

#Kommentare einzeln durchgehen und Metadaten plus Text ziehen
for comment in comments:
    comment_text = comment.find("div", class_="comment-text").text.strip()
    comment_author = comment.find("li", class_="comment-headline").text
    comment_time = comment.find("li", class_="comment-meta").text.strip()
    comment_time = re.sub(r"\s+"," ",comment_time).strip() 
    comment_time = re.sub(r'(\d+)\.(\d+)\.(\d+), (\d\d:\d\d) Uhr', r'\3-\2-\1T\4:00', comment_time)

    #Kommentare und Metadaten in XML schreiben
    comment_element = et.SubElement(comments_element, "comment", {"author":comment_author, "time":comment_time})
    comment_element.text = comment_text    

In [5]:
#XML formatieren und ausgeben
xmlstr = minidom.parseString(et.tostring(corpus)).toprettyxml(indent="  ")
print(xmlstr)

<?xml version="1.0" ?>
<corpus>
  <text title="Wer entscheidet über Sprache? » Die Engelbart-Galaxis » SciLogs - Wissenschaftsblogs" date="2021-12-13" author="Henning Lobin">
    <main>
      <head>Wer entscheidet über Sprache?</head>
      <p>Wenn ich am Ende dieses Jahres an die Diskussionen zur deutschen Sprache zurückdenke, die ich bei Medienauftritten und in Veranstaltungen geführt habe, dann ist dabei immer wieder eine ganz bestimmte Frage gestellt worden: Wer entscheidet eigentlich darüber, wie wir sprechen und schreiben, was wir sagen dürfen und was nicht? Wer hat die Entscheidungsbefugnis über die Aufnahme neuer Wörter ins Deutsche, über gendergerechte Sprache oder über Rechtschreibregeln?</p>
      <p>Wenn man eine Antwort auf diese Frage erhalten will, muss man differenzieren: Geht es um den Wortschatz oder die Grammatik, um mündliches oder schriftliches Deutsch, um die Orthografie? Für letztere kann man eine klare Antwort geben: Der Rat für deutsche Rechtschreibung berät üb